# Data Staging
GOURP 59

In [344]:
# !pip install pandas
import pandas as pd



# EXTRACT

In [345]:
# Read the csv file into a pandas dataframe
df= pd.read_csv('german_credit.csv')
outputFolder="DataMart/"

In [346]:
df

,checking_acc_status,duration,cred_hist,purpose,loan_amt,saving_acc_bonds,present_employment_since,installment_rate,personal_stat_gender,other_debtors_guarantors,...,property,age,other_installment_plans,housing,num_curr_loans,job,num_people_provide_maint,telephone,is_foreign_worker,target
0,below_0,6,risky_acc_or_curr_loan_other,radio_tv,1169,unknown_no_saving_acc,above_7y,4,male:single,none,...,real_estate,67,none,own,2,skilled_official,1,yes,yes,good
1,below_200,48,curr_loans_paid_duly,radio_tv,5951,below_100,below_4y,2,female:divorced_or_married,none,...,real_estate,22,none,own,1,skilled_official,1,no,yes,bad
2,no_cheking_acc,12,risky_acc_or_curr_loan_other,education,2096,below_100,below_7y,2,male:single,none,...,real_estate,49,none,own,1,unskilled_resident,2,no,yes,good
3,below_0,42,curr_loans_paid_duly,furniture_equipment,7882,below_100,below_7y,2,male:single,guarantor,...,life_insurance_or_aggreements,45,none,for_free,1,skilled_official,2,no,yes,good
4,below_0,24,delay_in_past,car_new,4870,below_100,below_4y,3,male:single,none,...,unknown_or_no_property,53,none,for_free,2,skilled_official,2,no,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no_cheking_acc,12,curr_loans_paid_duly,furniture_equipment,1736,below_100,below_7y,3,female:divorced_or_married,none,...,real_estate,31,none,own,1,unskilled_resident,1,no,yes,good
996,below_0,30,curr_loans_paid_duly,car_used,3857,below_100,below_4y,4,male:divorced,none,...,life_insurance_or_aggreements,40,none,own,1,management_or_self_emp,1,yes,yes,good
997,no_cheking_acc,12,curr_loans_paid_duly,radio_tv,804,below_100,above_7y,4,male:single,none,...,car_or_other,38,none,own,1,skilled_official,1,no,yes,good
998,below_0,45,curr_loans_paid_duly,radio_tv,1845,below_100,below_4y,4,male:single,none,...,unknown_or_no_property,23,none,for_free,1,skilled_official,1,yes,yes,bad


In [347]:
# Print a Series with the data type of each column
print(df.dtypes)

checking_acc_status         object
duration                     int64
cred_hist                   object
purpose                     object
loan_amt                     int64
saving_acc_bonds            object
present_employment_since    object
installment_rate             int64
personal_stat_gender        object
other_debtors_guarantors    object
present_residence_since      int64
property                    object
age                          int64
other_installment_plans     object
housing                     object
num_curr_loans               int64
job                         object
num_people_provide_maint     int64
telephone                   object
is_foreign_worker           object
target                      object
dtype: object


# TRANSFORMATION

Data cleaning

Check for null value

In [348]:
df=df[~df.isnull().any(axis=1)]

Seperate combined columns

In [349]:
df[['gender','marriage']]=df['personal_stat_gender'].str.split(':',expand=True)
df=df.drop('personal_stat_gender',axis=1)

Check and remove outliners or typo for important rows

In [350]:
# remove out range value in checking_acc_status column, there can only be one of these values : 'below_0', 'below_200', 'no_cheking_acc', 'above:200'
checking_account_values={'below_0', 'below_200', 'no_cheking_acc', 'above:200'}
df=df[df['checking_acc_status'].isin(checking_account_values)]

In [351]:
# remove out range value in duration, the value can only be positive integer
duration_values=df['duration']>0
df=df[duration_values]

In [352]:
# remove out range value in cred_hist column, there can only be one of these values : 'risky_acc_or_curr_loan_other' 'curr_loans_paid_duly' 'delay_in_past' 'no_loan_or_paid_duly_other' 'paid_duly_this_bank'
cred_hist_values={'risky_acc_or_curr_loan_other', 'curr_loans_paid_duly', 'delay_in_past', 'no_loan_or_paid_duly_other', 'paid_duly_this_bank'}
df=df[df['cred_hist'].isin(cred_hist_values)]

In [353]:
# remove out range value in purpose column
purpose_values=['radio_tv', 'education', 'furniture_equipment', 'car_new', 'car_used', 'business', 'domestic_applience', 'repairs', 'others', 'retraining']
df=df[df['purpose'].isin(purpose_values)]

In [354]:
# remove out range value in loan_amt column
df=df[df['loan_amt']>=0]

# remote out range value in saving_acc_bonds column
saving_acc_values=['unknown_no_saving_acc', 'below_100', 'below_1000', 'above_1000', 'below_500']
df=df[df['saving_acc_bonds'].isin(saving_acc_values)]

# remove out range value present_employment_since
present_employment_values=['above_7y', 'below_4y', 'below_7y', 'unemployed', 'below_1y']
df=df[df['present_employment_since'].isin(present_employment_values)]

# remove out range value installment_rate
installment_rate_values=[4, 2, 3, 1]
df=df[df['installment_rate'].isin(installment_rate_values)]


# remove out range value other_debtors_guarantors
other_debtors_guarantors_values=['none', 'guarantor', 'co_applicant']
df=df[df['other_debtors_guarantors'].isin(other_debtors_guarantors_values)]

# remove out range value present_residence_since
present_residence_since_values=[4, 2, 3, 1]
df=df[df['present_residence_since'].isin(present_residence_since_values)]

# remove out range value other_installment_plans	
other_installment_plans_values=['none', 'bank', 'store']
df=df[df['other_installment_plans'].isin(other_installment_plans_values)]

# remove out range value other_debtors_guarantors
other_debtors_guarantors_values=['none', 'guarantor', 'co_applicant']
df=df[df['other_debtors_guarantors'].isin(other_debtors_guarantors_values)]

# remove out range value present_residence_since
present_residence_since_values=[4, 2, 3, 1]
df=df[df['present_residence_since'].isin(present_residence_since_values)]

# remove out range value other_installment_plans
other_installment_plans_values=['none', 'bank', 'store']
df=df[df['other_installment_plans'].isin(other_installment_plans_values)]

# remove out range value housing
housing_values=['own', 'for_free', 'rent']
df=df[df['housing'].isin(housing_values)]


# remove out range value num_curr_loans
num_curr_loans_values=[2, 1, 3, 4]
df=df[df['num_curr_loans'].isin(num_curr_loans_values)]


# remove out range value job
job_values=['skilled_official', 'unskilled_resident', 'management_or_self_emp', 'unemployed_non_resident']
df=df[df['job'].isin(job_values)]

In [355]:
print(df.dtypes)
df

checking_acc_status         object
duration                     int64
cred_hist                   object
purpose                     object
loan_amt                     int64
saving_acc_bonds            object
present_employment_since    object
installment_rate             int64
other_debtors_guarantors    object
present_residence_since      int64
property                    object
age                          int64
other_installment_plans     object
housing                     object
num_curr_loans               int64
job                         object
num_people_provide_maint     int64
telephone                   object
is_foreign_worker           object
target                      object
gender                      object
marriage                    object
dtype: object


,checking_acc_status,duration,cred_hist,purpose,loan_amt,saving_acc_bonds,present_employment_since,installment_rate,other_debtors_guarantors,present_residence_since,...,other_installment_plans,housing,num_curr_loans,job,num_people_provide_maint,telephone,is_foreign_worker,target,gender,marriage
0,below_0,6,risky_acc_or_curr_loan_other,radio_tv,1169,unknown_no_saving_acc,above_7y,4,none,4,...,none,own,2,skilled_official,1,yes,yes,good,male,single
1,below_200,48,curr_loans_paid_duly,radio_tv,5951,below_100,below_4y,2,none,2,...,none,own,1,skilled_official,1,no,yes,bad,female,divorced_or_married
2,no_cheking_acc,12,risky_acc_or_curr_loan_other,education,2096,below_100,below_7y,2,none,3,...,none,own,1,unskilled_resident,2,no,yes,good,male,single
3,below_0,42,curr_loans_paid_duly,furniture_equipment,7882,below_100,below_7y,2,guarantor,4,...,none,for_free,1,skilled_official,2,no,yes,good,male,single
4,below_0,24,delay_in_past,car_new,4870,below_100,below_4y,3,none,4,...,none,for_free,2,skilled_official,2,no,yes,bad,male,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no_cheking_acc,12,curr_loans_paid_duly,furniture_equipment,1736,below_100,below_7y,3,none,4,...,none,own,1,unskilled_resident,1,no,yes,good,female,divorced_or_married
996,below_0,30,curr_loans_paid_duly,car_used,3857,below_100,below_4y,4,none,4,...,none,own,1,management_or_self_emp,1,yes,yes,good,male,divorced
997,no_cheking_acc,12,curr_loans_paid_duly,radio_tv,804,below_100,above_7y,4,none,4,...,none,own,1,skilled_official,1,no,yes,good,male,single
998,below_0,45,curr_loans_paid_duly,radio_tv,1845,below_100,below_4y,4,none,4,...,none,for_free,1,skilled_official,1,yes,yes,bad,male,single


# Create dimension table, generate surrogate key, load to csv files

In [356]:
# Create checking account dimension table, relate to fact table
checking_account_dimension=df['checking_acc_status'].unique()
checking_account_dimension=pd.DataFrame(checking_account_dimension,columns=['status'])
checking_account_dimension['surrogate key']=range(1,len(checking_account_dimension)+1)
checking_account_dimension=checking_account_dimension.reindex(columns=['surrogate key'] + [c for c in checking_account_dimension.columns if c!= 'surrogate key'])
checking_account_dimension.to_csv(outputFolder+'checking_account_dimension.csv', index=False)

df['checking_acc_status'] = df['checking_acc_status'].apply(lambda x: checking_account_dimension[checking_account_dimension['status'] == x]['surrogate key'].iloc[0])
df=df.rename(columns={'checking_acc_status':'checking account key'})

# Create credit dimension table, relate to fact table
credit_dimension=pd.DataFrame()
credit_dimension[['history','number of existing credit']]=df[['cred_hist','num_curr_loans']]
credit_dimension['surrogate key']=range(1,len(credit_dimension)+1)
credit_dimension=credit_dimension.reindex(columns= ['surrogate key'] + [c for c in credit_dimension.columns if c != 'surrogate key'])
credit_dimension.to_csv(outputFolder+'credit_dimension.csv', index=False)
df.insert(1,'credit key',range(1,1+len(credit_dimension)))
df.drop(columns=['cred_hist','num_curr_loans'], inplace=True)

# convert data type to boolean in good_risk column
df.rename(columns={'target':'good_risk'}, inplace=True)
mapping={'good':True, 'bad':False}
df['good_risk']=df['good_risk'].map(mapping)
df['good_risk'].astype(bool)

# create installment dimension and relate to fact table
installment_dimension=pd.DataFrame()
installment_dimension[['rate','other_installment_plans']]=df[['installment_rate','other_installment_plans']]
installment_dimension['surrogate key']=range(1,1+len(installment_dimension))
installment_dimension=installment_dimension.reindex(columns= ['surrogate key'] + [c for c in installment_dimension.columns if c != 'surrogate key'])
installment_dimension.to_csv(outputFolder+"installment_dimension.csv", index=False)
df.insert(2,'installment key',range(1,1+len(installment_dimension)))
df.drop(columns={'installment_rate','other_installment_plans'},inplace=True)

# create customer dimension and relate to fact table
customer_list=['gender','marriage','present_residence_since','age','telephone','present_employment_since','job','is_foreign_worker']
customer_dimension=df[customer_list]
customer_dimension=customer_dimension.rename(columns={'telephone':'has telephone', 'job':'job type'})
mapping={'yes':True, 'no':False}
customer_dimension['has telephone']=customer_dimension['has telephone'].map(mapping).astype(bool)
customer_dimension['is_foreign_worker']=customer_dimension['is_foreign_worker'].map(mapping).astype(bool)
customer_dimension['surrogate key']=range(1,1+len(customer_dimension))
customer_dimension=customer_dimension.reindex(columns= ['surrogate key'] + [c for c in customer_dimension.columns if c != 'surrogate key'])
customer_dimension.to_csv(outputFolder+'customer_dimension.csv', index=False)
df.drop(columns=['gender','marriage','present_residence_since','age','telephone','present_employment_since','job','is_foreign_worker'], inplace=True)
df.insert(3,'customer key',range(1,1+len(customer_dimension)))

# create purpose dimension, relate to fact table and load to csv
purpose_dimension=pd.DataFrame()
purpose_dimension['purpose']=df['purpose']
purpose_dimension.drop_duplicates(inplace=True)

# put the others option to the last
i=purpose_dimension[purpose_dimension['purpose'] == 'others'].index[0]
row=purpose_dimension.loc[i]
purpose_dimension=purpose_dimension.drop(i)
purpose_dimension.loc[len(purpose_dimension)]=row
purpose_dimension.insert(0,'surrogate key',range(1,1+len(purpose_dimension)))
purpose_dimension.reset_index(inplace=True,drop=True)
purpose_dimension.to_csv(outputFolder+"purpose_dimension.csv", index=False)
df['purpose']=df['purpose'].apply(lambda x : purpose_dimension[purpose_dimension['purpose'] ==x ]['surrogate key'].iloc[0])
df.insert(4, 'purpose key', df['purpose'])
df.drop(columns=['purpose'], inplace=True)

# create other guarantors dimension, relate to fact table and load to csv
other_guarantors_dimension=pd.DataFrame()
other_guarantors_dimension[['guarantor','number of people provide maintenance']]=df[['other_debtors_guarantors','num_people_provide_maint']]
other_guarantors_dimension.insert(0,'surrogate key',range(1,1+len(other_guarantors_dimension)))
other_guarantors_dimension.to_csv(outputFolder+'other_guarantors_dimension.csv', index=False)
df.insert(5,'other guarantors dimension key', range(1,1+len(other_guarantors_dimension)))
df.drop(columns=['other_debtors_guarantors','num_people_provide_maint'],inplace=True)

# create property dimension, relate to fact table, load to csv
property_dimension=pd.DataFrame()
property_dimension[['property','housing']]=df[['property','housing']]
property_dimension.insert(0,'surrogate key', range(1,1+len(property_dimension)))
property_dimension.to_csv(outputFolder+"property_dimension.csv", index=False)
df.insert(6, 'property key', range(1,1+len(property_dimension)))
df.drop(columns=['property','housing'], inplace=True)

# create saving account bonds dimension, realte to fact table, load to csv
saving_account_dimension=pd.DataFrame()
saving_account_dimension['savings']=df['saving_acc_bonds']
saving_account_dimension=saving_account_dimension.drop_duplicates()
saving_account_dimension.reset_index(inplace=True, drop=True)
saving_account_dimension.insert(0, "surrogate key", range(1,1+len(saving_account_dimension)))
saving_account_dimension.to_csv(outputFolder+"saving_account_dimension.csv", index=False)

df['saving_acc_bonds']=df['saving_acc_bonds'].apply(lambda x : saving_account_dimension[saving_account_dimension['savings'] == x]['surrogate key'].iloc[0])
df.insert(0,'saving account key', df['saving_acc_bonds'])
df.drop(columns=['saving_acc_bonds'], inplace=True)

# create loan fact table, generate primary key for it
df.insert(0, 'loan primary key', range(1,1+len(df)))
loan_fact_table=df
loan_fact_table.to_csv(outputFolder+"loan_fact_table.csv", index=False)
loan_fact_table




,loan primary key,saving account key,checking account key,credit key,installment key,customer key,purpose key,other guarantors dimension key,property key,duration,loan_amt,good_risk
0,1,1,1,1,1,1,1,1,1,6,1169,True
1,2,2,2,2,2,2,1,2,2,48,5951,False
2,3,2,3,3,3,3,2,3,3,12,2096,True
3,4,2,1,4,4,4,3,4,4,42,7882,True
4,5,2,1,5,5,5,4,5,5,24,4870,False
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2,3,996,996,996,3,996,996,12,1736,True
996,997,2,1,997,997,997,5,997,997,30,3857,True
997,998,2,3,998,998,998,1,998,998,12,804,True
998,999,2,1,999,999,999,1,999,999,45,1845,False
